In [ ]:
import cv2
import os
import numpy as np
import multiprocessing

# Paths
input_folder = "C:/Users/athar/Desktop/Minor/Data/p1"
output_folders = ["frame1", "frame2", "frame3"]

# Create output folders if they don't exist
for folder in output_folders:
    os.makedirs(folder, exist_ok=True)

# Global variables for ROI selection
frame_coords = []  # Stores (x1, y1, x2, y2) for each frame
refPt = []
selecting = False

def click_and_select(event, x, y, flags, param):
    """Handles mouse click and selection."""
    global refPt, selecting

    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        selecting = True

    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        selecting = False
        frame_coords.append((refPt[0][0], refPt[0][1], refPt[1][0], refPt[1][1]))
        cv2.rectangle(param, refPt[0], refPt[1], (0, 255, 0), 2)
        cv2.imshow("Select Frames", param)

def select_frames_manually(image_path):
    """Let user manually select three frames in the first image."""
    global frame_coords

    img = cv2.imread(image_path)
    clone = img.copy()
    
    cv2.namedWindow("Select Frames")
    cv2.setMouseCallback("Select Frames", click_and_select, param=clone)

    print("Select 3 regions (Click & Drag). Press 'q' when done.")
    while len(frame_coords) < 3:
        cv2.imshow("Select Frames", clone)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cv2.destroyAllWindows()
    return frame_coords

def process_image(image_path):
    """Extract selected frames and save them in folders."""
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to read {image_path}")
        return

    img_name = os.path.basename(image_path)

    for i, (x1, y1, x2, y2) in enumerate(frame_coords):
        frame = img[y1:y2, x1:x2]  # Extract region
        cv2.imwrite(os.path.join(output_folders[i], img_name), frame)

def main():
    """Run manual selection on the first image and apply to all."""
    image_paths = [os.path.join(input_folder, img) for img in os.listdir(input_folder) if img.endswith((".png", ".jpg", ".jpeg"))]

    if not image_paths:
        print("No images found!")
        return

    print("Select frames for the first image...")
    select_frames_manually(image_paths[0])  # Select manually

    print("Processing all images using selected frames...")
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        pool.map(process_image, image_paths)

if __name__ == "__main__":
    main()

# final

In [2]:
import cv2
import torch
import os
from ultralytics import YOLO
import time

# Load YOLOv8 model (pre-trained on COCO dataset)
model = YOLO("yolov8x.pt")

# COCO class IDs for vehicles
vehicle_classes = [2, 3, 5, 7]  # car, motorcycle, bus, truck

# Function to compute IoU
def compute_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Function to filter overlapping boxes
def filter_overlapping_boxes(boxes, iou_threshold=0.5):
    filtered_boxes = []
    for box in boxes:
        keep = True
        for filtered_box in filtered_boxes:
            if compute_iou(box.xyxy[0].tolist(), filtered_box.xyxy[0].tolist()) > iou_threshold:
                keep = False
                break
        if keep:
            filtered_boxes.append(box)
    return filtered_boxes

# Function to detect vehicles in an image and return count
def detect_vehicles(image_path):
    img = cv2.imread(image_path)
    results = model(img, conf=0.4, iou=0.5)[0]
    filtered_boxes = filter_overlapping_boxes(results.boxes)
    vehicle_count = sum(1 for box in filtered_boxes if int(box.cls[0]) in vehicle_classes)
    return vehicle_count

# Paths to image folders for each direction
image_folders = {
    'c1': "../Data/f1",
    'c2': "../Data/f2",
    'c3': "../Data/f3",
    'c4': "../Data/f4"
}

temp = 0
timer = 0
start1, start2, start3, start4 = 0, 30, 60, 90
t1, t2, t3, t4 = 30, 30, 30, 30
c1, c2, c3, c4 = 0, 0, 0, 0

while True:
    for i in range(1, t1 + 1):
        print(f"Timer: {i} | Direction: 1")
        if (start4 + t4 + 30) % 120 == i:
            c4 = detect_vehicles(os.path.join(image_folders['c4'], os.listdir(image_folders['c4'])[timer % 10]))
        if (start3 + t3 + 30) % 120 == i:
            c3 = detect_vehicles(os.path.join(image_folders['c3'], os.listdir(image_folders['c3'])[timer % 10]))
        if (start2 + t2 + 30) % 120 == i:
            c2 = detect_vehicles(os.path.join(image_folders['c2'], os.listdir(image_folders['c2'])[timer % 10]))
        # time.sleep(1)
        timer += 1
    
    for i in range(1, t2 + 1):
        print(f"Timer: {i} | Direction: 2")
        if (start4 + t4 + 30) % 120 == i:
            c4 = detect_vehicles(os.path.join(image_folders['c4'], os.listdir(image_folders['c4'])[timer % 10]))
        if (start3 + t3 + 30) % 120 == i:
            c3 = detect_vehicles(os.path.join(image_folders['c3'], os.listdir(image_folders['c3'])[timer % 10]))
        if (start1 + t1 + 30) % 120 == i:
            c1 = detect_vehicles(os.path.join(image_folders['c1'], os.listdir(image_folders['c1'])[timer % 10]))
        # time.sleep(1)
        timer += 1
    
    for i in range(1, t3 + 1):
        print(f"Timer: {i} | Direction: 3")
        if (start4 + t4 + 30) % 120 == i:
            c4 = detect_vehicles(os.path.join(image_folders['c4'], os.listdir(image_folders['c4'])[timer % 10]))
        if (start2 + t2 + 30) % 120 == i:
            c2 = detect_vehicles(os.path.join(image_folders['c2'], os.listdir(image_folders['c2'])[timer % 10]))
        if (start1 + t1 + 30) % 120 == i:
            c1 = detect_vehicles(os.path.join(image_folders['c1'], os.listdir(image_folders['c1'])[timer % 10]))
        # time.sleep(1)
        timer += 1
    
    for i in range( 1,t4 + 1):
        print(f"Timer: {i} | Direction: 4")
        if (start1 + t1 + 30) % 120 == i:
            c1 = detect_vehicles(os.path.join(image_folders['c1'], os.listdir(image_folders['c1'])[timer % 10]))
        if (start3 + t3 + 30) == i:
            c3 = detect_vehicles(os.path.join(image_folders['c3'], os.listdir(image_folders['c3'])[timer % 10]))
        if (start2 + t2 + 30) % 120 == i:
            c2 = detect_vehicles(os.path.join(image_folders['c2'], os.listdir(image_folders['c2'])[timer % 10]))
        # time.sleep(1)
        timer += 1
    
    total_vehicles = c1 + c2 + c3 + c4
    t1 = (c1 * 120) // total_vehicles if total_vehicles else 30
    t2 = (c2 * 120) // total_vehicles if total_vehicles else 30
    t3 = (c3 * 120) // total_vehicles if total_vehicles else 30
    t4 = (c4 * 120) // total_vehicles if total_vehicles else 30

    start1, start2, start3, start4 = 0, t1, t1 + t2 + 1, t1 + t2 + t3 + 1
    print(f"Updated Timers: {t1}, {t2}, {t3}, {t4}")


Timer: 1 | Direction: 1
Timer: 2 | Direction: 1
Timer: 3 | Direction: 1
Timer: 4 | Direction: 1
Timer: 5 | Direction: 1
Timer: 6 | Direction: 1
Timer: 7 | Direction: 1
Timer: 8 | Direction: 1
Timer: 9 | Direction: 1
Timer: 10 | Direction: 1
Timer: 11 | Direction: 1
Timer: 12 | Direction: 1
Timer: 13 | Direction: 1
Timer: 14 | Direction: 1
Timer: 15 | Direction: 1
Timer: 16 | Direction: 1
Timer: 17 | Direction: 1
Timer: 18 | Direction: 1
Timer: 19 | Direction: 1
Timer: 20 | Direction: 1
Timer: 21 | Direction: 1
Timer: 22 | Direction: 1
Timer: 23 | Direction: 1
Timer: 24 | Direction: 1
Timer: 25 | Direction: 1
Timer: 26 | Direction: 1
Timer: 27 | Direction: 1
Timer: 28 | Direction: 1
Timer: 29 | Direction: 1
Timer: 30 | Direction: 1

0: 352x640 4 cars, 1 truck, 1 traffic light, 877.3ms
Speed: 13.9ms preprocess, 877.3ms inference, 12.5ms postprocess per image at shape (1, 3, 352, 640)
Timer: 1 | Direction: 2
Timer: 2 | Direction: 2
Timer: 3 | Direction: 2
Timer: 4 | Direction: 2
Timer: 5 

KeyboardInterrupt: 

In [2]:
import cv2
import torch
import os
from ultralytics import YOLO
import time

In [4]:
# Load YOLOv8 model (pre-trained on COCO dataset)
model = YOLO("yolov8x.pt")

# COCO class IDs for vehicles
vehicle_classes = [2, 3, 5, 7]  # car, motorcycle, bus, truck

In [6]:
# Function to compute IoU
def compute_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou


In [8]:
# Function to filter overlapping boxes
def filter_overlapping_boxes(boxes, iou_threshold=0.5):
    filtered_boxes = []
    for box in boxes:
        keep = True
        for filtered_box in filtered_boxes:
            if compute_iou(box.xyxy[0].tolist(), filtered_box.xyxy[0].tolist()) > iou_threshold:
                keep = False
                break
        if keep:
            filtered_boxes.append(box)
    return filtered_boxes

In [10]:
# Function to detect vehicles in an image and return count
def detect_vehicles(image_path):
    img = cv2.imread(image_path)
    results = model(img, conf=0.4, iou=0.5)[0]
    filtered_boxes = filter_overlapping_boxes(results.boxes)
    vehicle_count = sum(1 for box in filtered_boxes if int(box.cls[0]) in vehicle_classes)
    return vehicle_count

In [12]:
# Paths to image folders for each direction
image_folders = {
    'c1': "../Data/f1",
    'c2': "../Data/f2",
    'c3': "../Data/f3",
    'c4': "../Data/f4"
}

In [14]:
temp = 0
timer = 0
start1, start2, start3, start4 = 0, 30, 60, 90
t1, t2, t3, t4 = 30, 30, 30, 30
c1, c2, c3, c4 = 0, 0, 0, 0

In [16]:
while True:
    for i in range(1, t1 + 1):
        print(f"Timer: {i} | Direction: 1")
        if (start4 + t4 + 30) % 120 == i:
            c4 = detect_vehicles(os.path.join(image_folders['c4'], os.listdir(image_folders['c4'])[timer % 10]))
        if (start3 + t3 + 30) % 120 == i:
            c3 = detect_vehicles(os.path.join(image_folders['c3'], os.listdir(image_folders['c3'])[timer % 10]))
        if (start2 + t2 + 30) % 120 == i:
            c2 = detect_vehicles(os.path.join(image_folders['c2'], os.listdir(image_folders['c2'])[timer % 10]))
        # time.sleep(1)
        timer += 1
    
    for i in range(1, t2 + 1):
        print(f"Timer: {i} | Direction: 2")
        if (start4 + t4 + 30) % 120 == i:
            c4 = detect_vehicles(os.path.join(image_folders['c4'], os.listdir(image_folders['c4'])[timer % 10]))
        if (start3 + t3 + 30) % 120 == i:
            c3 = detect_vehicles(os.path.join(image_folders['c3'], os.listdir(image_folders['c3'])[timer % 10]))
        if (start1 + t1 + 30) % 120 == i:
            c1 = detect_vehicles(os.path.join(image_folders['c1'], os.listdir(image_folders['c1'])[timer % 10]))
        # time.sleep(1)
        timer += 1
    
    for i in range(1, t3 + 1):
        print(f"Timer: {i} | Direction: 3")
        if (start4 + t4 + 30) % 120 == i:
            c4 = detect_vehicles(os.path.join(image_folders['c4'], os.listdir(image_folders['c4'])[timer % 10]))
        if (start2 + t2 + 30) % 120 == i:
            c2 = detect_vehicles(os.path.join(image_folders['c2'], os.listdir(image_folders['c2'])[timer % 10]))
        if (start1 + t1 + 30) % 120 == i:
            c1 = detect_vehicles(os.path.join(image_folders['c1'], os.listdir(image_folders['c1'])[timer % 10]))
        # time.sleep(1)
        timer += 1
    
    for i in range( 1,t4 + 1):
        print(f"Timer: {i} | Direction: 4")
        if (start1 + t1 + 30) % 120 == i:
            c1 = detect_vehicles(os.path.join(image_folders['c1'], os.listdir(image_folders['c1'])[timer % 10]))
        if (start3 + t3 + 30) == i:
            c3 = detect_vehicles(os.path.join(image_folders['c3'], os.listdir(image_folders['c3'])[timer % 10]))
        if (start2 + t2 + 30) % 120 == i:
            c2 = detect_vehicles(os.path.join(image_folders['c2'], os.listdir(image_folders['c2'])[timer % 10]))
        # time.sleep(1)
        timer += 1
    
    total_vehicles = c1 + c2 + c3 + c4
    t1 = (c1 * 120) // total_vehicles if total_vehicles else 30
    t2 = (c2 * 120) // total_vehicles if total_vehicles else 30
    t3 = (c3 * 120) // total_vehicles if total_vehicles else 30
    t4 = (c4 * 120) // total_vehicles if total_vehicles else 30

    start1, start2, start3, start4 = 0, t1, t1 + t2 + 1, t1 + t2 + t3 + 1
    print(f"Updated Timers: {t1}, {t2}, {t3}, {t4}")

Timer: 1 | Direction: 1
Timer: 2 | Direction: 1
Timer: 3 | Direction: 1
Timer: 4 | Direction: 1
Timer: 5 | Direction: 1
Timer: 6 | Direction: 1
Timer: 7 | Direction: 1
Timer: 8 | Direction: 1
Timer: 9 | Direction: 1
Timer: 10 | Direction: 1
Timer: 11 | Direction: 1
Timer: 12 | Direction: 1
Timer: 13 | Direction: 1
Timer: 14 | Direction: 1
Timer: 15 | Direction: 1
Timer: 16 | Direction: 1
Timer: 17 | Direction: 1
Timer: 18 | Direction: 1
Timer: 19 | Direction: 1
Timer: 20 | Direction: 1
Timer: 21 | Direction: 1
Timer: 22 | Direction: 1
Timer: 23 | Direction: 1
Timer: 24 | Direction: 1
Timer: 25 | Direction: 1
Timer: 26 | Direction: 1
Timer: 27 | Direction: 1
Timer: 28 | Direction: 1
Timer: 29 | Direction: 1
Timer: 30 | Direction: 1

0: 352x640 4 cars, 1 truck, 1 traffic light, 858.5ms
Speed: 12.1ms preprocess, 858.5ms inference, 14.5ms postprocess per image at shape (1, 3, 352, 640)
Timer: 1 | Direction: 2
Timer: 2 | Direction: 2
Timer: 3 | Direction: 2
Timer: 4 | Direction: 2
Timer: 5 

KeyboardInterrupt: 

In [ ]:
import cv2
import torch
import os
from ultralytics import YOLO
import time

# Load YOLOv8 model (pre-trained on COCO dataset)
model = YOLO("yolov8x.pt")

# COCO class IDs for vehicles
vehicle_classes = [2, 3, 5, 7]  # car, motorcycle, bus, truck

# Function to compute IoU
def compute_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

# Function to filter overlapping boxes
def filter_overlapping_boxes(boxes, iou_threshold=0.5):
    filtered_boxes = []
    for box in boxes:
        keep = True
        for filtered_box in filtered_boxes:
            if compute_iou(box.xyxy[0].tolist(), filtered_box.xyxy[0].tolist()) > iou_threshold:
                keep = False
                break
        if keep:
            filtered_boxes.append(box)
    return filtered_boxes

# Function to detect vehicles in an image and return count
def detect_vehicles(image_path):
    img = cv2.imread(image_path)
    results = model(img, conf=0.4, iou=0.5)[0]
    filtered_boxes = filter_overlapping_boxes(results.boxes)
    vehicle_count = sum(1 for box in filtered_boxes if int(box.cls[0]) in vehicle_classes)
    return vehicle_count

# Paths to image folders for each direction
image_folders = {
    'c1': "../Data/f1",
    'c2': "../Data/f2",
    'c3': "../Data/f3",
    'c4': "../Data/f4"
}

temp = 0
timer = 0
start1, start2, start3, start4 = 0, 30, 60, 90
t1, t2, t3, t4 = 30, 30, 30, 30
c1, c2, c3, c4 = 0, 0, 0, 0

while True:
    for i in range(1, t1 + 1):
        print(f"Timer: {i} | Direction: 1")
        if (start4 + t4 + 30) % 120 == i:
            c4 = detect_vehicles(os.path.join(image_folders['c4'], os.listdir(image_folders['c4'])[timer % 10]))
        if (start3 + t3 + 30) % 120 == i:
            c3 = detect_vehicles(os.path.join(image_folders['c3'], os.listdir(image_folders['c3'])[timer % 10]))
        if (start2 + t2 + 30) % 120 == i:
            c2 = detect_vehicles(os.path.join(image_folders['c2'], os.listdir(image_folders['c2'])[timer % 10]))
        # time.sleep(1)
        timer += 1
    
    for i in range(1, t2 + 1):
        print(f"Timer: {i} | Direction: 2")
        if (start4 + t4 + 30) % 120 == i:
            c4 = detect_vehicles(os.path.join(image_folders['c4'], os.listdir(image_folders['c4'])[timer % 10]))
        if (start3 + t3 + 30) % 120 == i:
            c3 = detect_vehicles(os.path.join(image_folders['c3'], os.listdir(image_folders['c3'])[timer % 10]))
        if (start1 + t1 + 30) % 120 == i:
            c1 = detect_vehicles(os.path.join(image_folders['c1'], os.listdir(image_folders['c1'])[timer % 10]))
        # time.sleep(1)
        timer += 1
    
    for i in range(1, t3 + 1):
        print(f"Timer: {i} | Direction: 3")
        if (start4 + t4 + 30) % 120 == i:
            c4 = detect_vehicles(os.path.join(image_folders['c4'], os.listdir(image_folders['c4'])[timer % 10]))
        if (start2 + t2 + 30) % 120 == i:
            c2 = detect_vehicles(os.path.join(image_folders['c2'], os.listdir(image_folders['c2'])[timer % 10]))
        if (start1 + t1 + 30) % 120 == i:
            c1 = detect_vehicles(os.path.join(image_folders['c1'], os.listdir(image_folders['c1'])[timer % 10]))
        # time.sleep(1)
        timer += 1
    
    for i in range( 1,t4 + 1):
        print(f"Timer: {i} | Direction: 4")
        if (start1 + t1 + 30) % 120 == i:
            c1 = detect_vehicles(os.path.join(image_folders['c1'], os.listdir(image_folders['c1'])[timer % 10]))
        if (start3 + t3 + 30) == i:
            c3 = detect_vehicles(os.path.join(image_folders['c3'], os.listdir(image_folders['c3'])[timer % 10]))
        if (start2 + t2 + 30) % 120 == i:
            c2 = detect_vehicles(os.path.join(image_folders['c2'], os.listdir(image_folders['c2'])[timer % 10]))
        # time.sleep(1)
        timer += 1
    
    total_vehicles = c1 + c2 + c3 + c4
    t1 = (c1 * 120) // total_vehicles if total_vehicles else 30
    t2 = (c2 * 120) // total_vehicles if total_vehicles else 30
    t3 = (c3 * 120) // total_vehicles if total_vehicles else 30
    t4 = (c4 * 120) // total_vehicles if total_vehicles else 30

    start1, start2, start3, start4 = 0, t1, t1 + t2 + 1, t1 + t2 + t3 + 1
    print(f"Updated Timers: {t1}, {t2}, {t3}, {t4}")


Timer: 1 | Direction: 1
Timer: 2 | Direction: 1
Timer: 3 | Direction: 1
Timer: 4 | Direction: 1
Timer: 5 | Direction: 1
Timer: 6 | Direction: 1
Timer: 7 | Direction: 1
Timer: 8 | Direction: 1
Timer: 9 | Direction: 1
Timer: 10 | Direction: 1
Timer: 11 | Direction: 1
Timer: 12 | Direction: 1
Timer: 13 | Direction: 1
Timer: 14 | Direction: 1
Timer: 15 | Direction: 1
Timer: 16 | Direction: 1
Timer: 17 | Direction: 1
Timer: 18 | Direction: 1
Timer: 19 | Direction: 1
Timer: 20 | Direction: 1
Timer: 21 | Direction: 1
Timer: 22 | Direction: 1
Timer: 23 | Direction: 1
Timer: 24 | Direction: 1
Timer: 25 | Direction: 1
Timer: 26 | Direction: 1
Timer: 27 | Direction: 1
Timer: 28 | Direction: 1
Timer: 29 | Direction: 1
Timer: 30 | Direction: 1

0: 352x640 4 cars, 1 truck, 1 traffic light, 831.3ms
Speed: 2.8ms preprocess, 831.3ms inference, 1.6ms postprocess per image at shape (1, 3, 352, 640)
Timer: 1 | Direction: 2
Timer: 2 | Direction: 2
Timer: 3 | Direction: 2
Timer: 4 | Direction: 2
Timer: 5 | 